In [ ]:
class MaxHeap:
    """
    A class that implements properties and methods supporting a max priority queue data structure.

    Attributes
    ----------
    heap : list
        A Python list where task objects are stored
    heap_size: int
        An integer counter of the number of tasks present in the max heap
    """

    def __init__(self):
        """
        Initializes an instance of MaxHeap.
        """
        self.heap = []        # Initialize an empty list to store tasks
        self.heap_size = 0    # Initialize the heap size counter

    def left(self, i):
        """
        Takes the index of the parent node and returns the index of the left child node.

        Parameters
        ----------
        i: int
            Index of parent node

        Returns
        -------
        int
            Index of the left child node
        """
        # Formula for left child
        return 2 * i + 1

    def right(self, i):
        """
        Takes the index of the parent node and returns the index of the right child node.

        Parameters
        ----------
        i: int
            Index of parent node

        Returns
        -------
        int
            Index of the right child node
        """
        # Formula for right child
        return 2 * i + 2

    def parent(self, i):
        """
        Takes the index of the child node and returns the index of the parent node.

        Parameters
        ----------
        i: int
            Index of child node

        Returns
        -------
        int
            Index of the parent node
        """
        # Formula for the parent
        return (i - 1) // 2

    def max_key(self):
        """
        Returns the task with the highest priority in the priority queue.

        Parameters
        ----------
        None

        Returns
        -------
        Task
            The task object with the highest priority
        """
        return self.heap[0]

    def heappush(self, task):
        """
        Insert a task into the priority queue.

        Parameters
        ----------
        task: Task
            The task object to be inserted

        Returns
        -------
        None
        """
        self.heap.append(task)                    # Add the new task to the end of the list
        self.increase_key(self.heap_size, task)  # Fix the heap to maintain the max-heap property
        self.heap_size += 1                     # Increment the heap size

    def increase_key(self, i, task):
        """
        Modifies the value of a task in a max priority queue with a higher value.

        Parameters
        ----------
        i: int
            The index of the task to be modified
        task: Task
            The new task object

        Returns
        -------
        None
        """
        if task < self.heap[i]:
            raise ValueError('New task has lower priority than the current task')
        self.heap[i] = task
        
        # Fix the max-heap property by swapping with the parent if necessary
        while i > 0 and self.heap[self.parent(i)] < self.heap[i]:
            j = self.parent(i)
            holder = self.heap[j]
            self.heap[j] = self.heap[i]
            self.heap[i] = holder
            i = j

    def heapify(self, i):
        """
        Creates a max heap from the index given.

        Parameters
        ----------
        i: int
            The index of the root node of the subtree to be heapified

        Returns
        -------
        None
        """
        l = self.left(i)
        r = self.right(i)
        heap = self.heap
        
        # Determine the largest element among the root, left, and right children
        if l <= (self.heap_size - 1) and heap[l] > heap[i]:
            largest = l
        else:
            largest = i
        if r <= (self.heap_size - 1) and heap[r] > heap[largest]:
            largest = r
            
        # If the largest element is not the root, swap with the root and recursively heapify
        if largest != i:
            heap[i], heap[largest] = heap[largest], heap[i]
            self.heapify(largest)

    def heappop(self):
        """
        Returns the task with the highest priority in the max priority queue and removes it.

        Parameters
        ----------
        None

        Returns
        -------
        Task
            The task object with the highest priority
        """
        if self.heap_size < 1:
            raise ValueError('Heap underflow: There are no tasks in the priority queue')
        max_task = self.heap[0]
        self.heap[0] = self.heap[-1]   # Swap the last element with the root
        self.heap.pop()                # Remove the last element
        self.heap_size -= 1            # Decrement the heap size
        
        # Fix the max-heap property starting from the root
        self.heapify(0)
        
        return max_task

In [ ]:
class Task:
    def __init__(self, id, description, duration, dependencies=None, status="N", urgency=False, importance=False, start_time=None):
        """
        Initialize a Task object.

        Parameters
        ----------
        id : int
            Unique identifier for the task.
        description : str
            Description of the task.
        duration : int
            Duration of the task in minutes.
        dependencies : list, optional
            List of task IDs that this task depends on.
        status : str, optional
            Current status of the task (default is "N" for not started).
        urgency : bool, optional
            Indicates the urgency of the task.
        importance : bool, optional
            Indicates the importance of the task.
        start_time : int, optional
            Scheduled start time for the task.
        """
        self.id = id
        self.description = description
        self.duration = duration
        self.dependencies = dependencies or []
        self.status = status
        self.urgency = urgency
        self.importance = importance
        self.start_time = start_time
        self.priority = int

        
    def __lt__(self, other):
        """
        Compare two Task objects based on priority.

        Parameters
        ----------
        other : Task
            Another Task object for comparison.

        Returns
        -------
        bool
            True if this task has lower priority than the other, False otherwise.
        """
        # Checking if the the two comparables is of the same type
        if isinstance(other, Task):
            
            # Prioritizing comparision by the start_time
            if self.start_time is not None and other.start_time is not None:
                return self.start_time > other.start_time
            
            # Comparing usign priority value
            return self.priority < other.priority
        else:
            raise TypeError("Cannot compare Task with object of type {}".format(type(other)))

            
            
class MyScheduler:
    
    
    # Possible status of a task
    NOT_STARTED = 'N'
    IN_PRIORITY_QUEUE = 'I'
    COMPLETED = 'C'

    
    def __init__(self, tasks, initial_starting_time: int, final_time: int):
        """
        Initialize a MyScheduler object.

        Parameters
        ----------
        tasks : list of Task
            List of Task objects to be scheduled.
        initial_starting_time : int
            Initial time for scheduling.
        final_time : int
            Final time for scheduling.
        """
        self.tasks = tasks
        self.priority_queue = MaxHeap()
        self.priority_queue_fixed = MaxHeap()
        self.starting_time_queue = MaxHeap()
        self.priority = float()
        self.current_time = initial_starting_time
        self.final_time = final_time
        

    def compute_dependency_weight(self, comparable):
        """
        Compute the weight of task dependencies.

        Parameters
        ----------
        comparable : Task
            The task for which to compute the dependency weight.

        Returns
        -------
        int
            The computed dependency weight.
        """
        weight = 0
        
        # Making sure that even if the comparable's weight is updated in case it is the final task 
        # after the series of its dependencies
        if comparable.dependencies:
            weight += 900 * len(comparable.dependencies)
        for task in self.tasks:
            if task != comparable:
                if comparable.id in task.dependencies:
                    # Prioritizing the dependees of tasks associated with a start_time
                    if task.start_time:
                        weight += 100000
                    weight += 900
        return weight

    
    def compute_priority(self):
        """
        Compute priority for all tasks based on importance, urgency, dependency weight, and duration
        by updating the attribute task.priority.
=
        """
        #temporarily storing the tasks in another list
        updated_priority_tasks = self.tasks
        
        for task in updated_priority_tasks:
            
            #formula for computing priority
            task.priority = int(task.importance) * 100 + int(task.urgency) * 50 + self.compute_dependency_weight(task) - task.duration/100
            
        # Updating back the tasks make sure that the updating process dosen't interfere 
        # witht the other loops within the compute_dependency_weight method
        self.tasks = updated_priority_tasks

        
    def calculate_time_window(self, current_time):
        """
        Calculate the available time window for scheduling based on the presence of a fixed time task
        or in its abscnece the interval between the the current time and the final time.

        Parameters
        ----------
        current_time : int
            The current time in the scheduler.

        Returns
        -------
        int
            The available time window.
        """
        if self.starting_time_queue.heap_size < 1:
            return self.final_time - current_time
        else:
            return self.starting_time_queue.max_key().start_time - current_time

        
    def is_dependent_duration(self, task):
        """
        Calculate the total duration of a task and its dependencies so we don't pick a task to fill the time window 
        when it the task is only meaningful in sequence with its other dependencies

        Parameters
        ----------
        task : Task
            The task for which to calculate the duration including dependencies.

        Returns
        -------
        int
            The total duration of the task and its dependencies.
        """
        tasks_together = []
        duration = 0
        for t in self.tasks:
            if (task.id in t.dependencies) and (t.id not in tasks_together) and (not t.start_time):
                duration += t.duration
        return duration + task.duration

    
    def get_tasks_ready(self):
        """
        Add tasks to the starting_time_queue and priority queues if they are ready for scheduling.
        """
        for task in self.tasks:
            
            # Filtering by the not started status
            if task.status == self.NOT_STARTED:
                
                # Declaring a bool to control for the pushing into the starting_time_queue
                # if the same fixed time task has already been pushed
                insertion = True
                
                # Searching for the task inside the queue to not push it again
                for i in self.starting_time_queue.heap:
                    if i == task:
                        insertion = False
                
                # Pushing the task in case it is not present in the starting_time_queue
                if insertion and task.start_time:
                    self.starting_time_queue.heappush(task)
                
                # Filtering by depndencies
                if not task.dependencies:
                    
                    #Pushing the tasks with a fixed time
                    if task.start_time is not None:
                        self.priority_queue_fixed.heappush(task)
                        task.status = self.IN_PRIORITY_QUEUE
                        
                    #Pushing flexibale tasks
                    else:
                        self.priority_queue.heappush(task)
                        task.status = self.IN_PRIORITY_QUEUE

                        
    def remove_dependency(self, task_id):
        """
        Remove a task from the dependencies of other tasks.

        Parameters
        ----------
        task_id : int
            The ID of the task to be removed from dependencies.
        """
        for t in self.tasks:
            if t.id != task_id and task_id in t.dependencies:
                t.dependencies.remove(task_id)

                
    def format_time(self, time):
        """
        Format time for display.

        Parameters
        ----------
        time : int
            The time to be formatted.

        Returns
        -------
        str
            The formatted time string.
        """
        return f"{(time//60)%24}:{time%60:02d}"

        
    def check_unscheduled_tasks(self):
        """
        Check if there are unscheduled tasks or tasks stuck in the priority queue.

        Returns
        -------
        bool
            True if there are unscheduled tasks, False otherwise.
        """
        return any(task.status == self.NOT_STARTED or task.status == self.IN_PRIORITY_QUEUE for task in self.tasks)

    
    def run(self, initial_starting_time: int, printable = True):
        """
        Run the scheduling algorithm.

        Parameters
        ----------
        initial_starting_time : int
            Initial time for scheduling.
        printable : bool, optional
            Whether to print outputs (default is True).

        Returns
        -------
        list
            List of scheduled task IDs.
        """
        if printable:
            print("Tasks added to the simple scheduler:")
            print("--------------------------------------")
        
        current_time = self.current_time
        self.compute_priority()
        total_time = 0
        schedule = []

        while self.check_unscheduled_tasks() and current_time < self.final_time:
            
            # Getting tasks ready in their respective queues
            self.get_tasks_ready()
            
            #calculating the time window based on the updated current_time
            time_window = self.calculate_time_window(current_time)
            
            # Initializing a bool to break from the while loop if a task has been scheduled
            # This allows for updated and accurate 
            flexible_scheduler = False
            
            while self.priority_queue and self.priority_queue.heap_size > 0 and not flexible_scheduler:
                
                # Extracting the root element from priority_queue
                task = self.priority_queue.heappop()
                
                if task.duration <= time_window and self.is_dependent_duration(task) <= time_window:
                    
                    if printable:
                        # Printing the task
                        print(f"🕰t={self.format_time(current_time)}")
                        print(f"\tstarted '{task.description}' for {task.duration} mins... with {task.priority} priority")
                        print(f"\t✅ t={self.format_time(current_time+task.duration)}, task completed!")
                    
                    # Updating the time variables
                    current_time += task.duration
                    time_window = self.calculate_time_window(current_time)
                    total_time += task.duration
                    
                    self.remove_dependency(task.id)
                    task.status = self.COMPLETED
                    schedule.append(task.id)
                    
                    flexible_scheduler = True
                    
                else:
                    task.status = self.NOT_STARTED
                


            if self.priority_queue_fixed and self.priority_queue_fixed.heap_size > 0 and self.priority_queue_fixed.max_key().start_time >= current_time:
                
                # Extracting the root element from priority_queue_fixed
                task = self.priority_queue_fixed.heappop()
                
                # Extracting the root element from starting_time_queue also to make sure that both queues match
                task_keeping_track = self.starting_time_queue.heappop()
                
                if printable:
                    # Printing the task
                    print(f"🕰t={self.format_time(current_time)}")
                    print(f"\tstarted '{task.description}' for {task.duration} mins... with {task.priority} priority")
                    print(f"\t✅ t={self.format_time(current_time+task.duration)}, task completed!")
                
                # Updating the time variables
                current_time += task.duration
                total_time += task.duration
                
                task.status = self.COMPLETED
                self.remove_dependency(task.id)
                schedule.append(task.id)
                
        if printable:    
            print(f"\n🏁 Completed all planned tasks in {total_time//60}h{total_time%60:02d}min!")
            print("Final Schedule: ",schedule)

In [ ]:
# Example tasks
task_0 = Task(id=0, description='Wake Up ', duration=0, dependencies=None, urgency=True, importance=True, start_time=480)
task_1 = Task(id=1, description='Wearing running shoes', duration=10, dependencies=None, urgency=False, importance=True)
task_2 = Task(id=2, description='Running around Namsan Park', duration=30, dependencies=[1], urgency=False, importance=True)
task_3 = Task(id=3, description='Calling Family', duration=10, dependencies=None, urgency=True, importance=False)
task_4 = Task(id=4, description='Checking Phone', duration=30, dependencies=None, urgency=False, importance=False)
task_5 = Task(id=5, description='Attending CS110 Class', duration=90, dependencies=[11], urgency=True, importance=True, start_time=760)  # Fixed start time for CS110 Class
task_6 = Task(id=6, description='Changing to Gym clothes', duration=5, dependencies=None, urgency=False, importance=True)
task_7 = Task(id=7, description='Commuting to the Gym', duration=5, dependencies=[6], urgency=False, importance=True)
task_8 = Task(id=8, description='Working out in the Gym', duration=60, dependencies=[6, 7], urgency=False, importance=True)
task_9 = Task(id=9, description='Eating Korean Fried chicken with friends', duration=120, dependencies=None, urgency=False, importance=True)
task_10 = Task(id=10, description='Walking back home while exploring Haebangchon', duration=30, dependencies=[9], urgency=False, importance=True)
task_11 = Task(id=11, description='Doing PCW', duration=260, dependencies=None, urgency=True, importance=True)

my_tasks = [task_0, task_1, task_2, task_3, task_4, task_5, task_6, task_7, task_8, task_9, task_10, task_11]

# Example scheduler
simple_scheduler = MyScheduler(my_tasks, initial_starting_time=480, final_time=24*60)
simple_scheduler.run(my_tasks)


In [ ]:
import random
from copy import deepcopy
# Example tasks
task_0 = Task(id=0, description='Wake Up ', duration=0, dependencies=None, urgency=True, importance=True, start_time=480)
task_1 = Task(id=1, description='Wearing running shoes', duration=10, dependencies=None, urgency=False, importance=True)
task_2 = Task(id=2, description='Running around Namsan Park', duration=30, dependencies=[1], urgency=False, importance=True)
task_3 = Task(id=3, description='Calling Family', duration=10, dependencies=None, urgency=True, importance=False)
task_4 = Task(id=4, description='Checking Phone', duration=30, dependencies=None, urgency=False, importance=False)
task_5 = Task(id=5, description='Attending CS110 Class', duration=90, dependencies=[11], urgency=True, importance=True, start_time=760)  # Fixed start time for CS110 Class
task_6 = Task(id=6, description='Changing to Gym clothes', duration=5, dependencies=None, urgency=False, importance=True)
task_7 = Task(id=7, description='Commuting to the Gym', duration=5, dependencies=[6], urgency=False, importance=True)
task_8 = Task(id=8, description='Working out in the Gym', duration=60, dependencies=[6, 7], urgency=False, importance=True)
task_9 = Task(id=9, description='Eating Korean Fried chicken with friends', duration=120, dependencies=None, urgency=True, importance=True)
task_10 = Task(id=10, description='Walking back home while exploring Haebangchon', duration=30, dependencies=[9], urgency=True, importance=True)
task_11 = Task(id=11, description='Doing PCW', duration=260, dependencies=None, urgency=True, importance=True)

my_tasks = [task_0, task_1, task_2, task_3, task_4, task_5, task_6, task_7, task_8, task_9, task_10, task_11]


tasks_shuffled = deepcopy(my_tasks)  # Make a copy to avoid modifying the original list
random.shuffle(tasks_shuffled)

simple_scheduler_shuffled = MyScheduler(tasks_shuffled, initial_starting_time=480, final_time=24*60)
simple_scheduler_shuffled.run(tasks_shuffled)



# Comparing the two schedulers for equality
# assert simple_scheduler.run(my_tasks) == simple_scheduler_shuffled.run(tasks_shuffled)

In [ ]:
import random
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def generate_random_tasks(num_tasks):
    tasks = []
    for i in range(num_tasks):
        max_dependencies = min(i, 3)
        dependencies =  random.sample(range(i), random.randint(0, min(5, i)))
        
        task = Task(
            id=i,
            description=f'Task {i}',
            duration=random.randint(30, 120),
            dependencies=dependencies,
            status="N",
            urgency=bool(random.getrandbits(1)),
            importance=bool(random.getrandbits(1)),
            start_time=None
        )
        tasks.append(task)
    return tasks

# Function to run the scheduling algorithm with experimental analysis
def run_experiment(num_tasks):
    tasks = generate_random_tasks(num_tasks)
    initial_starting_time = 0
    final_time = 1440  # 24 hours in minutes

    scheduler = MyScheduler(tasks, initial_starting_time, final_time)

    start_time = time.time()
    schedule = scheduler.run(initial_starting_time, printable=False)
    end_time = time.time()

    execution_time = end_time - start_time

    

    return execution_time

# Function to run multiple experiments and calculate running average time
def run_multiple_experiments(num_tasks_list, num_experiments):
    average_execution_times = []

    for num_tasks in num_tasks_list:
        execution_times = []
        for _ in range(num_experiments):
            execution_time = run_experiment(num_tasks)
            execution_times.append(execution_time)

        average_execution_time = sum(execution_times) / num_experiments
        
        average_execution_times.append(average_execution_time)

    return average_execution_times

def perform_linear_regression(x, y):
    fit = np.polyfit(x, y, 2)
    return fit

# Define the list of num_tasks to experiment with
num_tasks_list = list(range(1, 16))  # above 16 and it breaks

# Run experiments and collect average execution times
num_experiments = 500
average_execution_times = run_multiple_experiments(num_tasks_list, num_experiments)

# Perform linear regression
fit = perform_linear_regression(num_tasks_list, average_execution_times)

# Plotting the results
plt.plot(num_tasks_list, average_execution_times, marker='o')
plt.plot(num_tasks_list, np.polyval(fit, num_tasks_list), '--', color='red', label='Best Fit Line')
plt.xlabel('Number of Tasks')
plt.ylabel('Average Execution Time (seconds)')
plt.title('Average Execution Time for Different Number of Tasks')
plt.grid(True)
plt.show()

